In [2]:
!nvidia-smi

Fri Mar  7 20:28:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

# temporary fix for some weird locale bug
import locale
locale.getpreferredencoding = lambda: "UTF-8"

HOME: /content


In [4]:
%cd {HOME}
!git clone https://github.com/hkchengrex/Grounded-Segment-Anything
%env CUDA_HOME=/usr/local/cuda
%env BUILD_WITH_CUDA=True
%env AM_I_DOCKER=False
%cd {HOME}/Grounded-Segment-Anything
!pip uninstall -y GroundingDINO
!pip install -e GroundingDINO
!pip install -q -e segment_anything

/content
fatal: destination path 'Grounded-Segment-Anything' already exists and is not an empty directory.
env: CUDA_HOME=/usr/local/cuda
env: BUILD_WITH_CUDA=True
env: AM_I_DOCKER=False
/content/Grounded-Segment-Anything
Found existing installation: groundingdino 0.1.0
Uninstalling groundingdino-0.1.0:
  Successfully uninstalled groundingdino-0.1.0
Obtaining file:///content/Grounded-Segment-Anything/GroundingDINO
  Preparing metadata (setup.py) ... done
  Running setup.py develop for groundingdino
  Preparing metadata (setup.py) ... done


In [5]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

# temporary fix for some weird locale bug
import locale
locale.getpreferredencoding = lambda: "UTF-8"

HOME: /content/Grounded-Segment-Anything


In [6]:
# either one of these should work
%cd {HOME}
try:
  import groundingdino
  from groundingdino.util.inference import Model as GroundingDINOModel
except ImportError:
  import GroundingDINO
  from GroundingDINO.groundingdino.util.inference import Model as GroundingDINOModel

/content/Grounded-Segment-Anything


In [7]:
%cd {HOME}
!git clone https://github.com/hkchengrex/Tracking-Anything-with-DEVA
%cd {HOME}/Tracking-Anything-with-DEVA
!pip install -q -e .

/content/Grounded-Segment-Anything
Cloning into 'Tracking-Anything-with-DEVA'...
remote: Enumerating objects: 552, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 552 (delta 70), reused 85 (delta 55), pack-reused 436 (from 1)
Receiving objects: 100% (552/552), 1.09 MiB | 3.43 MiB/s, done.
Resolving deltas: 100% (297/297), done.
/content/Grounded-Segment-Anything/Tracking-Anything-with-DEVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Building editable for deva (pyproject.toml) ... done


In [8]:
%cd {HOME}/Tracking-Anything-with-DEVA
!wget -q -P ./saves/ https://github.com/hkchengrex/Tracking-Anything-with-DEVA/releases/download/v1.0/DEVA-propagation.pth
!wget -q -P ./saves/ https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
!wget -q -P ./saves/ https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!wget -q -P ./saves/ https://github.com/hkchengrex/Tracking-Anything-with-DEVA/releases/download/v1.0/GroundingDINO_SwinT_OGC.py

/content/Grounded-Segment-Anything/Tracking-Anything-with-DEVA


In [ ]:
%cd {HOME}/Tracking-Anything-with-DEVA

import os
from os import path
from argparse import ArgumentParser

import torch
import numpy as np

from deva.model.network import DEVA
from deva.inference.inference_core import DEVAInferenceCore
from deva.inference.result_utils import ResultSaver
from deva.inference.eval_args import add_common_eval_args, get_model_and_config
from deva.inference.demo_utils import flush_buffer
from deva.ext.ext_eval_args import add_ext_eval_args, add_text_default_args
from deva.ext.grounding_dino import get_grounding_dino_model
from deva.ext.with_text_processor import process_frame_with_text as process_frame

from tqdm import tqdm
import json

torch.autograd.set_grad_enabled(False)

# for id2rgb
np.random.seed(42)

# default parameters
parser = ArgumentParser()
add_common_eval_args(parser)
add_ext_eval_args(parser)
add_text_default_args(parser)

# load model and config
args = parser.parse_args([])
cfg = vars(args)
cfg['enable_long_term'] = True

# Load our checkpoint
deva_model = DEVA(cfg).cuda().eval()
if args.model is not None:
    model_weights = torch.load(args.model)
    deva_model.load_weights(model_weights)
else:
    print('No model loaded.')

gd_model, sam_model = get_grounding_dino_model(cfg, 'cuda')

/content/Grounded-Segment-Anything/Tracking-Anything-with-DEVA


final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p {HOME}/data
!cp "/content/drive/MyDrive/rgb_video.mp4" "/content/data/rgb_video.mp4"

In [74]:
cfg['enable_long_term_count_usage'] = True
cfg['max_num_objects'] = 50
cfg['size'] = 480
cfg['DINO_THRESHOLD'] = 0.3
cfg['amp'] = True
cfg['chunk_size'] = 4
cfg['detection_every'] = 3
cfg['max_missed_detection_count'] = 10
cfg['sam_variant'] = 'original'
cfg['temporal_setting'] = 'online' # semionline usually works better; but online is faster for this demo
cfg['pluralize'] = True
cfg['prompt'] = 'window.door.kitchen_object'
category_mask_thresholds = [0.5, 0.5, 0.1]

!rm -rf {HOME}/data/masks
SOURCE_VIDEO_PATH = f"{HOME}/data/rgb_video.mp4"
OUTPUT_VIDEO_PATH = f"{HOME}/data/output.webm"
MASK_DIR = f"{HOME}/data/masks"
os.makedirs(MASK_DIR, exist_ok=True)

In [ ]:
%cd {HOME}/Tracking-Anything-with-DEVA

from deva.ext.with_text_processor import process_frame_with_text as process_frame_text
import tempfile
import cv2

deva = DEVAInferenceCore(deva_model, config=cfg)
deva.next_voting_frame = cfg['num_voting_frames'] - 1
deva.enabled_long_id()

# obtain temporary directory
result_saver = ResultSaver(None, None, dataset='gradio', object_manager=deva.object_manager)
writer_initizied = False

cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
ti = 0
# only an estimate
with torch.cuda.amp.autocast(enabled=cfg['amp']):
    with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))) as pbar:
        while (cap.isOpened()):
            ret, frame = cap.read()
            if ret == True:
                if not writer_initizied:
                    h, w = frame.shape[:2]
                    writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'vp80'), fps, (w, h))
                    writer_initizied = True
                    result_saver.writer = writer

                process_frame_text(deva, gd_model, sam_model, 'null.png', result_saver, ti, image_np=frame)
                prob = deva.last_mask[0]
                category_masks = [None, None, None]

                tmp_to_obj = deva.object_manager.tmp_id_to_obj
                for tmp_id in range(prob.shape[0]):
                  if tmp_id in tmp_to_obj:
                    obj_info = tmp_to_obj[tmp_id]
                    category_id = obj_info.vote_category_id()
                    obj_prob = prob[tmp_id].cpu().numpy()
                    binary_mask = (obj_prob > 0.7).astype(np.uint8) * 255

                    if category_id is not None and category_id < len(category_mask_thresholds):
                      if category_masks[category_id] is None:
                        category_masks[category_id] = np.zeros_like(binary_mask)
                      category_masks[category_id] = np.maximum(category_masks[category_id], binary_mask)

                for category_id, category_mask in enumerate(category_masks):
                  if category_mask is not None:
                    cv2.imwrite(os.path.join(MASK_DIR, f"frame_{ti:05d}_category_{category_id}.png"), category_mask)

                ti += 1
                pbar.update(1)
            else:
                break
    flush_buffer(deva, result_saver)
writer.release()
cap.release()
deva.clear_buffer()

/content/Tracking-Anything-with-DEVA


  4%|▍         | 9/201 [00:05<01:32,  2.07it/s]

In [1]:
!ls {HOME}/data/masks
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.imshow(plt.imread(f'{HOME}/data/masks/frame_00004_category_0.png'))
plt.axis('off')
plt.show()

ls: cannot access '{HOME}/data/masks': No such file or directory


NameError: name 'HOME' is not defined

<Figure size 1000x1000 with 0 Axes>